# Search for keywords in Google patents

## Goal

We wish to search for a list of keywords in Google patents for presence and absence. The input file contains the US patents as rows and keywords as columns. We would return a matrix filled with presence and absence after searching is completed.

## Dependencies

After searching for "Google patent scrapers", a few Python libraries look promising that we'll experiment here.
- [PyPatent](https://pypi.org/project/pypatent/)
- [google-patent-scraper](https://pypi.org/project/google-patent-scraper/)

In [1]:
!pip install -U google-patent-scraper lxml XlsxWriter

Requirement already up-to-date: google-patent-scraper in /Users/bao/miniconda3/lib/python3.7/site-packages (1.0.5)
Requirement already up-to-date: lxml in /Users/bao/miniconda3/lib/python3.7/site-packages (4.5.0)
Requirement already up-to-date: XlsxWriter in /Users/bao/miniconda3/lib/python3.7/site-packages (1.2.8)


## Input

In [2]:
import pandas as pd
import os
import re
import os.path as op
import sys
from bs4 import BeautifulSoup as bs4
from google_patent_scraper import scraper_class
PUB = "Publication Number"

CACHEDIR = "_cache"
os.makedirs(CACHEDIR, exist_ok=True)

In [3]:
df = pd.read_excel("data.xlsx")
df.head()

,Publication Number,malonyl-CoA,aspartate,beta-alanine,malonate semialdehyde,malonyl-coa reductase,lactoyl-CoA,dehydratase,decarboxylase,dehydrogenase,hydrolyase,byproduct,tolerance,purification
0,US20150072399A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US20150072384A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US20150064754A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US20150056684A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US20150056651A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from multiprocessing import Lock, Process, Queue

lock = Lock() # The lock is global

def worker(i, patent, total_patents, queue):
    cache_html = op.join(CACHEDIR, "{}.html".format(patent))
    progress = "[{}/{}]".format(i + 1, total_patents)
    # Download webpage
    if op.exists(cache_html):
        message = "Cache exists for `{}` {}".format(patent, progress)
        with open(cache_html) as fp: 
            soup = bs4(fp.read(), features="lxml")
    else:
        message = "Cache patent `{}` {}".format(patent, progress)
        scraper = scraper_class()
        err, soup, url = scraper.request_single_patent(patent)
        with open(cache_html, "w") as fw:
            print(str(soup), file=fw)

    with lock:
        print(message, file=sys.stderr)        
    
    # Search for keywords
    for keyword in keywords:
        keyword_pat = re.compile(keyword.strip(), re.IGNORECASE)
        keyword_count = len(soup.findAll(text=keyword_pat))
        queue.put((i, keyword, keyword_count))

keywords = df.columns[1:]
total_patents = df.shape[0]
processes = []
results_queue = Queue()
for i, row in df.iterrows():
    patent = row[PUB]
    worker(i, patent, total_patents, results_queue)
    break
    
for i, row in df.iterrows():
    if i == 0: # not sure why we need to run a non-parallel function to 'prime' this run
        continue
    patent = row[PUB]
    p = Process(target=worker, args=(i, patent, total_patents, results_queue))
    p.start()
    processes.append(p)

for p in processes:
    p.join()

while not results_queue.empty():
    i, patent, patent_count = results_queue.get()
    df.at[i, patent] = patent_count

print("Success! All {} processes completed".format(len(processes) + 1), file=sys.stderr) 

Cache exists for `US20150072399A1` [1/13]
Cache exists for `US8741624` [12/13]
Cache exists for `US8829112` [10/13]
Cache exists for `US20140309451A1` [9/13]
Cache exists for `US8894762` [8/13]
Cache exists for `US8911978` [7/13]
Cache exists for `US20140135526A1` [13/13]
Cache exists for `US20140230091A1` [11/13]
Cache exists for `US20150064754A1` [3/13]
Cache exists for `US20140377823A1` [6/13]
Cache exists for `US20150072384A1` [2/13]
Cache exists for `US20150056684A1` [4/13]
Cache exists for `US20150056651A1` [5/13]
Success! All 13 processes completed


In [5]:
mf = df.astype(dict((keyword, "int32") for keyword in keywords))
mf.head()

,Publication Number,malonyl-CoA,aspartate,beta-alanine,malonate semialdehyde,malonyl-coa reductase,lactoyl-CoA,dehydratase,decarboxylase,dehydrogenase,hydrolyase,byproduct,tolerance,purification
0,US20150072399A1,14,4,5,10,8,0,4,7,113,0,0,10,19
1,US20150072384A1,0,0,5,12,0,0,1,60,35,0,1,32,22
2,US20150064754A1,76,4,3,27,52,0,0,6,97,0,2,8,5
3,US20150056684A1,14,7,8,9,9,0,1,20,29,0,0,178,16
4,US20150056651A1,207,9,9,15,36,0,7,32,59,0,2,161,30


In [6]:
output = "export.xlsx"
writer = pd.ExcelWriter(output, engine="xlsxwriter")
mf.to_excel(writer, sheet_name="Sheet1", startrow=1, header=False, index=False)

# Styling
workbook = writer.book
worksheet = writer.sheets["Sheet1"]

header_format = workbook.add_format({
    'bold': True,
    'valign': 'top',
    'fg_color': '#D7E4BC',
    'border': 1})

link_format = workbook.add_format({
    'font_color': 'red',
    'bold':       1,
    'underline':  1,
})

# Write the column headers with the defined format
for col_num, value in enumerate(df.columns.values):
    worksheet.write(0, col_num, value, header_format)

def make_hyperlink(value):
    url = "https://patents.google.com/patent/{}"
    return url.format(value)
    
# Format URL link
for row_num, value in enumerate(df[PUB]):
    worksheet.write_url(row_num + 1, 0, make_hyperlink(value), link_format, string=value)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

print("Success! Output written to `{}`".format(output), file=sys.stderr)

Success! Output written to `export.xlsx`
